# Step 1

Run this to create the function that translate the MULTIPOLYGON string from geo-network to MultiPolygon array in GeoJson

In [5]:
import re

In [7]:
original = "MULTIPOLYGON (((102.0 2.0, 103.0 2.0, 103.0 3.0, 102.0 3.0, 102.0 2.0)), ((100.0 0.0, 101.0 0.0, 101.0 1.0, 100.0 1.0, 100.0 0.0), (100.2 0.2, 100.8 0.2, 100.8 0.8, 100.2 0.8, 100.2 0.2)))"

In [8]:
elements = get_elements(original)
elements

['(102.0 2.0, 103.0 2.0, 103.0 3.0, 102.0 3.0, 102.0 2.0)',
 '(100.0 0.0, 101.0 0.0, 101.0 1.0, 100.0 1.0, 100.0 0.0), (100.2 0.2, 100.8 0.2, 100.8 0.8, 100.2 0.8, 100.2 0.2)']

In [3]:
def get_elements(original: str) -> list:
    new_str = re.sub(r'\)\)$', '', original.replace('MULTIPOLYGON ((', ''))
    if ')), ((' in new_str:
        new_str = new_str.replace(')), ((', ');; (')
        elements = new_str.split(';; ')
    else:
        elements = []
        elements.append(new_str)
    return elements

def translate_coordinates(element: str) -> list:
    mp = re.sub(r'^\(', '', element)
    mp = re.sub(r'\)$', '', mp)
    coors = mp.split(",")
    ar = []
    for coor in coors:
        ar.append([float(i) for i in coor.strip().split(" ")])
    return ar

def translate_trajectory_multipolygon(original: str) -> list:
    elements = get_elements(original)
    results = []
    for i, element in enumerate(elements):
        processed_ar = []
        if '), (' in element:
            sub_elements = element.split('), (')
            for sub_element in sub_elements:
                processed_ar.append(translate_coordinates(sub_element))
        else:
            processed_ar.append(translate_coordinates(element))
        results.append(processed_ar)
    return results


results = translate_trajectory_multipolygon(original)
results

NameError: name 're' is not defined

In [10]:
# Some init setting, replace the API key for your instance

import urllib.request as request
import requests as postApi
import json

headers = {
    "Content-Type": "application/json",
    "Authorization": "ApiKey WTlTdUNZVUJUWGtsWTg4N1Z2T1Q6cFFadkFicGdSbFNnbVdPRkNrcWVKdw=="
}


In [14]:
# Create the correct Mapping, the key here is to map the geoPolygon to geo_shape, the elastic do not guess the type correct and
# hence need to override it.
x = postApi.put('https://ec2-54-253-84-18.ap-southeast-2.compute.amazonaws.com:9200/search-soop-trajectory-data', headers=headers, json={
    "settings": {
        "index": {
            "number_of_shards": "1",
            "analysis": {
                "filter": {
                    "front_ngram": {
                        "type": "edge_ngram",
                        "min_gram": "1",
                        "max_gram": "12"
                    },
                    "bigram_joiner": {
                        "max_shingle_size": "2",
                        "token_separator": "",
                        "output_unigrams": "false",
                        "type": "shingle"
                    },
                    "bigram_max_size": {
                        "type": "length",
                        "max": "16",
                        "min": "0"
                    },
                    "en-stem-filter": {
                        "name": "light_english",
                        "type": "stemmer",
                        "language": "light_english"
                    },
                    "bigram_joiner_unigrams": {
                        "max_shingle_size": "2",
                        "token_separator": "",
                        "output_unigrams": "true",
                        "type": "shingle"
                    },
                    "delimiter": {
                        "split_on_numerics": "true",
                        "generate_word_parts": "true",
                        "preserve_original": "false",
                        "catenate_words": "true",
                        "generate_number_parts": "true",
                        "catenate_all": "true",
                        "split_on_case_change": "true",
                        "type": "word_delimiter_graph",
                        "catenate_numbers": "true",
                        "stem_english_possessive": "true"
                    },
                    "en-stop-words-filter": {
                        "type": "stop",
                        "stopwords": "_english_"
                    }
                },
                "analyzer": {
                    "i_prefix": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "front_ngram"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    },
                    "iq_text_delimiter": {
                        "filter": [
                            "delimiter",
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "en-stop-words-filter",
                            "en-stem-filter"
                        ],
                        "type": "custom",
                        "tokenizer": "whitespace"
                    },
                    "q_prefix": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    },
                    "iq_text_base": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "en-stop-words-filter"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    },
                    "iq_text_stem": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "en-stop-words-filter",
                            "en-stem-filter"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    },
                    "i_text_bigram": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "en-stem-filter",
                            "bigram_joiner",
                            "bigram_max_size"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    },
                    "q_text_bigram": {
                        "filter": [
                            "cjk_width",
                            "lowercase",
                            "asciifolding",
                            "en-stem-filter",
                            "bigram_joiner_unigrams",
                            "bigram_max_size"
                        ],
                        "type": "custom",
                        "tokenizer": "standard"
                    }
                }
            },
            "number_of_replicas": "1"
        }
    },
    "mappings": {
        "dynamic": "true",
        "dynamic_templates": [
            {
                "all_text_fields": {
                    "match_mapping_type": "string",
                    "mapping": {
                        "analyzer": "iq_text_base",
                        "fields": {
                            "delimiter": {
                                "analyzer": "iq_text_delimiter",
                                "type": "text",
                                "index_options": "freqs"
                            },
                            "joined": {
                                "search_analyzer": "q_text_bigram",
                                "analyzer": "i_text_bigram",
                                "type": "text",
                                "index_options": "freqs"
                            },
                            "prefix": {
                                "search_analyzer": "q_prefix",
                                "analyzer": "i_prefix",
                                "type": "text",
                                "index_options": "docs"
                            },
                            "enum": {
                                "ignore_above": 2048,
                                "type": "keyword"
                            },
                            "stem": {
                                "analyzer": "iq_text_stem",
                                "type": "text"
                            }
                        }
                    }
                }
            },
            {
                "goe_array_as_shape": {
                    "match_mapping_type": "*",
                    "match": "geoPolygon",
                    "mapping": { "type" : "geo_shape"}
                }
            }
        ]
    }
}, verify=False);

print(x.text)

/home/vhnguyen/anaconda3/envs/architecturereview/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ec2-54-253-84-18.ap-southeast-2.compute.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{"acknowledged":true,"shards_acknowledged":true,"index":"search-soop-trajectory-data"}


In [11]:
# This is the url to get catalog into, we want to loop all dataset in catalog, call API to import to app search engine and inject the json doc
# we need to modify the geoPolygon so that the format match
api = "https://catalogue-imos.aodn.org.au/geonetwork/srv/eng/q?any={uuid}&type=dataset&_content_type=json&fast=index&from={f}&resultType=details&sortBy=relevance&to={t}"

with request.urlopen(api.format(uuid='ca15915b-38c5-4e95-8d25-9cc42c4dd485', f=1, t=1)) as r:
    obj = json.loads(r.read().decode("utf-8"))
    totalRecord = int(obj["summary"]["@count"])

# Loop all record
for i in range(1, totalRecord + 1, 100) :
    with request.urlopen(api.format(uuid='ca15915b-38c5-4e95-8d25-9cc42c4dd485', f=i, t=i+1)) as r:
        obj = json.loads(r.read().decode("utf-8"))
        metas = obj["metadata"]

        if not isinstance(metas, (list)):
            metas = [ metas ]

        for c in metas:
            c["geoPolygon"] = {
                "type": "MultiPolygon",
                "coordinates": translate_trajectory_multipolygon(c["geoPolygon"])
            }

            
            print(c)
            
#             x = postApi.post(
#                 "https://ec2-54-253-84-18.ap-southeast-2.compute.amazonaws.com:9200/search-soop-trajectory-data/_doc",
#                 json = c,
#                 verify=False,
#                 headers=headers
#             )

#             print(x.text)


{'title': 'IMOS - SOOP-Sea Surface Temperature (SST) Sub-facility - Near real-time data', 'abstract': "The Sea Surface Temperature (SST) sub-facility aims to enable accurate, quality controlled, SST data to be supplied in near real-time (within 24 hours) from SOOPs and research vessels in oceans worldwide. Remotely sensed sea surface temperature (SST) data is an important input to ocean, numerical weather prediction, seasonal and climate models. In order to improve calibration and validation of satellite SST in the Australian region, there is a need for high quality in situ SST observations with greater timeliness, spatial and temporal coverage than is currently available. Regions particularly lacking in moored or drifting buoy observations are the Western Pacific Tropical Warm Pool region (Indonesia), close to the Australian coast (including Bass Strait) and the Southern Ocean. Current ship SST observations from ships of opportunity (SOOP) are either of questionable accuracy or diffic